In [1]:
import pandas as pd
import numpy as np
import math
import re
from ipywidgets import widgets,interact
from IPython.display import display
import IPython
from IPython.display import display_html 
display_html("""<button onclick="$('.input, .prompt, .output_stderr, .output_error').toggle();">Toggle Code</button>""", raw=True)

Toggle Code

In [2]:
m_text=widgets.FloatText(
description='总质量（kg）',
value=16000
)
high_text=widgets.FloatText(
description='外廓高（mm）',
value=2500
)
width_text=widgets.FloatText(
description='外廓宽（mm）',
value=3540
)
r_text=widgets.FloatText(
description='轮胎半径（mm）',
value=508
)
Cd_text=widgets.FloatText(
description='风阻系数',
value='0.5'
)
eff_trans_text=widgets.FloatText(
description='传动效率',
value='0.9127'
)
g_text=widgets.FloatText(
description='重力加速度',
value='9.8'
)
lamata_text=widgets.FloatText(
description='转动惯量系数',
value='1.14'
)
motore_text=widgets.Text(
description='电机额定功率/转速/扭矩',
value='120/1140/1005'
)
motorz_text=widgets.Text(
description='电机峰值功率/转速/扭矩',
value='250/2800/2800'
)
transmission=widgets.Text(
description='变速箱速比',
value='1.00'
)
reducer=widgets.FloatText(
description='主减速比',
value=6.33
)

In [3]:
def viehicle_power(m_full,high,width,r,Cd,eff_trans,g,lamata,motor_e,motor_z,i1,i0):
    k=re.compile('\d+')
    motor_e=k.findall(motor_e)
    Pe=float(motor_e[0])
    Te=float(motor_e[2])
    motor_z=k.findall(motor_z)
    Pz=float(motor_z[0])
    nz=float(motor_z[1])
    Tz=float(motor_z[2])
    h=re.compile('\d+\.\d+')
    try:
        i1=h.findall(i1)
        i1=np.array(list(map(float,i1)))
    except:
        i1=float(i1)
    A=width/1000*high/1000*0.81
    r=r/1000
    i0=float(i0)
    i1=i1*i0
    def vehicle_res(v,grade):
        f=0.0076+0.000056*v
        Ff=f*m_full*g*math.cos(math.atan(grade))
        Fw=Cd*A*v**2/21.15
        Fj=m_full*g*math.sin(math.atan(grade))
        Fres=Ff+Fw+Fj
        return Fres
    def vehicle_F(v,rate,peak):
        v=np.array(v)
        rate=np.array(rate)
        if peak=='peak':
            P=Pz
            T=Tz
        else:
            P=Pe
            T=Te
        n=v*rate/0.377/r
        T1=P*9550/n*eff_trans
        T1=np.array(T1)
        T1[T1>T]=T
        F=T1*rate/r
        return F
    
    #车速坡道车速爬坡度计算
    count=0
    grade_z=0
    v_max=np.zeros(len(i1))
    v_maxz=np.zeros(len(i1))
    v_4=np.zeros(len(i1))
    v_15=np.zeros(len(i1))
    for i_current in i1:
        v=np.arange(0,200,0.01)
        Fi1=vehicle_F(v,i_current,'rated')-vehicle_res(v,0)
        for i in range(1,len(v)):
            if Fi1[i]<0 or v[i]*i_current/0.377/r>nz:
                v_max[count]=v[i-1]
                break
            
        Fi1=vehicle_F(v,i_current,'peak')-vehicle_res(v,0)
        for i in range(1,len(v)):
            if Fi1[i]<0 or v[i]*i_current/0.377/r>nz:
                v_maxz[count]=v[i-1]
                break
        
        v=np.arange(0.1,200,0.01)
        Fi2=vehicle_F(v,i_current,'rated')-vehicle_res(v,0.04)
        for i in range(1,len(v)):
            if Fi2[i]<0 or v[i]*i_current/0.377/r>nz:
                v_4[count]=v[i-1]
                break
        
        v=np.arange(0,200,0.01) 
        Fi3=vehicle_F(v,i_current,'peak')-vehicle_res(v,0.15)
        for i in range(1,len(v)):
            if Fi3[i]<0 or v[i]*i_current/0.377/r>nz:
                v_15[count]=v[i-1]
                break
        count=count+1
    for grade in np.arange(1,0.04,-0.001):
        Fi4=vehicle_F(5,i1.max(),'peak')-vehicle_res(5,grade)
        if Fi4>0:
            grade_z=grade
            break
    print('持续最高车速为:%skm/h'%(v_max.max()))
    print('持续最高车速为:%skm/h'%(v_maxz.max()))
    print('最大爬坡度:%s%%'%(grade_z*100))
    print('4%%坡道车速为:%skm/h'%(v_4.max()))
    print('15%%坡道车速为:%skm/h'%(v_15.max()))
    
    #加速时间计算   
    error_count=0
    v=0
    t=0
    dt=0.1
    flag30=1
    flag50=1
    flag80=1
    Fi5=np.zeros(len(i1))
    t_30=np.nan
    t_50=np.nan
    t_80=np.nan
    while v<100 :
        for i in range(0,len(i1)):
            Fi5[i]=vehicle_F(v,i1[i],'peak')-vehicle_res(v,0)
        a=Fi5.max()/lamata/m_full
        v=v+a*dt*3.6
        t=t+dt
        error_count=error_count+1
        if v>30 and flag30==1:
            t_30=t
            flag30=0
        if v>50 and flag50==1:
            t_50=t
            flag50=0
        if v>80 and flag80==1:
            t_80=t
            flag80=0
        if error_count>10000:
            break
    t_30=round(t_30,2)
    t_50=round(t_50,2)
    t_80=round(t_80,2)
    print('0-30km/h加速时间:%s\n0-50km/h加速时间:%s\n0-80km/h加速时间:%s'%(t_30,t_50,t_80))
    #return grade_z,v_max.max(),v_maxz.max(),v_4.max(),v_15.max(),t_30,t_50,t_80


In [4]:
interact(viehicle_power,m_full=m_text,high=high_text,width=width_text,r=r_text,Cd=Cd_text,eff_trans=eff_trans_text,g=g_text,lamata=lamata_text,motor_e=motore_text,motor_z=motorz_text,i1=transmission,i0=reducer)

A Jupyter Widget

<function __main__.viehicle_power>